In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import base64
import json
import logging
import os
import re
import smtplib
import sys
import time
import threading
from copy import deepcopy
from functools import partial
from itertools import chain, combinations, product
from numbers import Number
from pathlib import Path
from uuid import uuid4
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser
from scipy.optimize import minimize
from scipy.interpolate import CubicSpline
from sklearn.decomposition import PCA
from sklearn import set_config

from api.risk.var import VaRPnl, get_db_from_clarion_positions, get_fut_trade_spec, get_var_ts, get_db_from_specs, var_ptf, get_clarion_fut_trade_spec, cvar_ptf
from api.email import smtp_server, to_email_test, to_email_all, df_to_html, from_email
from api.instruments.future import FutureChain

from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods, parse_swap_period
from gioutils.tools_clarion import get_clarion_positions

from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
    SQL,
    parse_irb_remote_args, 
    tsh_to_series
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    parse_dates,
    today,
    get_generic_future, 
    get_specific_future,
    get_bbg_fut_chain_ticker,
    get_root,
    encode_df_as_image_tag
)

from techi.api import MACD, BBANDS, RSI, DX, ADXR, ADX, STOCH, WILLR, SAR

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)
set_config(transform_output = "pandas")
rng = np.random.default_rng(7)

In [4]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-05-16 14:55:43,972 - Authenticating...
2023-05-16 14:55:44,699 - Successfully authenticated


In [5]:
ticker = 'AAPL US Equity'
bar_len = 1  # minutes 
start_datetime = f"{pd.Timestamp(2023, 1, 1):%Y-%m-%dT%H:%M:%S}"
end_datetime = f"{pd.Timestamp.today():%Y-%m-%dT%H:%M:%S}"

In [6]:
bars = bq.bdib(
    ticker, 
    'TRADE', 
    bar_len, 
    start_datetime=start_datetime,
    end_datetime=end_datetime,
).set_index('time')

In [7]:
from techi.api2 import MACD

In [9]:
macd = MACD()

In [ ]:
def agg_bars(bars, freq, dropna=True):
    bars_agg = bars.resample(freq).agg(
        {
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum',
            'numEvents': 'sum',
            'value': 'sum',
        }
    )
    return bars_agg if not dropna else bars_agg.dropna()

In [ ]:
bars5 = bq.bdib(
    ticker, 
    'TRADE', 
    5, 
    start_datetime=start_datetime,
    end_datetime=end_datetime,
).set_index('time')

In [ ]:
bars_1d = agg_bars(bars, 'B')

In [ ]:
bars_1d

In [ ]:
ind = ADXR
ind = DX

In [ ]:
ind(bars_1d).dropna()

In [ ]:
bars_1d

In [ ]:
ind

In [ ]:
bars_agg = bars.resample('5min').agg(
    {
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
        'numEvents': 'sum',
        'value': 'sum',
    }
)

In [ ]:
x_breaks = bars.index.to_series().groupby(pd.Grouper(freq='d')).agg(['last', 'first']).dropna()
x_breaks['last'] = x_breaks['last'] + pd.tseries.offsets.Minute(bar_len)
x_breaks['first'] = x_breaks['first'].shift(-1)
x_breaks = x_breaks.dropna()
x_breaks_list = [dict(bounds=[f"{row.iloc[0]}", f"{row.iloc[1]}"]) for index, row in x_breaks.iterrows()]

In [ ]:
fig = go.Figure(
    data=[
        go.Candlestick(
            x=p.index,
            open=p['open'],
            high=p['high'],
            low=p['low'],
            close=p['close'],
            increasing_line_width=1,
            decreasing_line_width=1
        )
    ]
).update_layout(
    height=500,
    template='plotly_dark',
    paper_bgcolor='#1e222d',
    plot_bgcolor='#1e222d',
    margin=dict(t=10, b=10),
    xaxis_rangeslider_visible=False,
    xaxis_rangebreaks=[
        *x_breaks_list,
        dict(
            bounds=["sat", "mon"]
        ),
    ]
)

In [ ]:
super_clarion(clarion)
runner = Runner(clarion)

In [ ]:
generator = 'usd sofr'

In [ ]:
get_generic_future(['sfrm4', 'sfrh5', 'sfrz5'], bq=bq)

In [ ]:
get_generic_future(['edu4', 'edm5', 'edh6'], bq=bq)

In [ ]:
default_expiries = ['0y']

default_tenors = [
    # '1w',
    # '2w',
    '1m',
    '2m',
    '3m',
    '4m',
    '5m',
    '6m',
    '7m',
    '8m',
    '9m',
    '10m',
    '11m',
    '12m',
    '18m',
    '2y',
    '3y',
    '4y',
    '5y',
    '6y',
    '7y',
    '8y',
    '9y',
    '10y',
    '11y',
    '12y',
    '15y',
    '20y',
    '25y',
    '30y',
    '35y',
    '40y',
    '50y',
]

In [ ]:
default_expiries_tenors = [
    ('1y', '1y'),
    ('2y', '1y'),
    ('3y', '1y'),
    ('4y', '1y'),
    ('5y', '1y'),
    ('6y', '1y'),
    ('7y', '1y'),
    ('8y', '1y'),
    ('9y', '1y'),
    ('10y', '1y'),
    ('11y', '1y'),
    ('12y', '1y'),
    ('13y', '1y'),
    ('14y', '1y'),
    ('15y', '5y'),
    ('20y', '5y'),
    ('25y', '5y'),
    ('30y', '10y'),
    ('40y', '10y'),
    ('50y', '10y'),
    ('60y', '10y')
]

In [ ]:
class IRBTRemote:
    
    metrics = [
        'Rate',
        'Strike',
        'PV',
        'Premium',
        'Vol',
        'Ann01',
    ]
    
    def __init__(self):
        pass
        
    def get_data(self, data, start_date=None, end_date=None, hist='2y'):
        self.ref = data
        
        if not end_date:
            end_date = pd.Timestamp.today().floor('d')

        start_date, end_date = parse_dates(start_date, end_date, hist)
        
        
        db = bh.bhDbRange(
            id='irtb',
            range=data.T.reset_index(),  # .iloc[:, -1:]
            headers=True,
            transpose=True,
            namode='o'
        )
        
        res = bh.bhTsFiRemoteRiskCalc(
            tradesdb=db,
            pblevals=self.metrics,
            startdate=start_date,
            enddate=end_date,
            irbtdbname=SQL.FITradePricer.db,
            irbtdbsrvname=SQL.FITradePricer.server,
            remoteservername=SQL.IRBTServer.server,
            remoteserverport=SQL.IRBTServer.port,
            timeout=180
        )
        self.res = res 
        df_res = {}
        for index, row in res.py.query('Status == "OK"').sort_values('TradeId').iterrows():
            df_res[row['TradeId'], row['Metric'].lower()] = tsh_to_series(row['TS'])
        self.hist = pd.DataFrame(df_res)

In [ ]:
def get_irbt_curve_data_from_product(generator, expiries=None, tenors=None):
    expiries = expiries or default_expiries
    tenors = tenors or default_tenors
    
    data = [
        parse_irb_remote_args(
            trade_id=i,
            ccy=generator.split(' ')[0].strip(),
            generator=generator,
            instrument='irs',
            eff_exp=per[0],
            tenor_mty=per[1],
            pay_receive='pay'
        ) for i, per in enumerate(product(expiries, tenors))
    ]
    data = pd.DataFrame(data)
    return data

In [ ]:
def get_irbt_curve_data_from_tuples(generator, expiries_tenors=None):
    expiries_tenors = expiries_tenors or default_expiries_tenors
    data = [
        parse_irb_remote_args(
            trade_id=i,
            ccy=generator.split(' ')[0].strip(),
            generator=generator,
            instrument='irs',
            eff_exp=per[0],
            tenor_mty=per[1],
            pay_receive='pay'
        ) for i, per in enumerate(expiries_tenors)
    ]
    data = pd.DataFrame(data)
    return data

In [ ]:
irbt = IRBTRemote()

In [ ]:
irbt.get_data(
    data=get_irbt_curve_data_from_tuples(
        generator=generator
    ),
    hist='20y'
)

In [ ]:
info = irbt.ref[[
    'TradeId',
    'EffExp',
    'MtyTnr',
    'TradeLabel',
]]

In [ ]:
info = info.assign(
    EffExp=pd.Categorical(info['EffExp'], categories=info['EffExp'].drop_duplicates(), ordered=True),
    MtyTnr=pd.Categorical(info['MtyTnr'], categories=info['MtyTnr'].drop_duplicates(), ordered=True),
    expiry_days=[round(((today() + parse_offset(offset)) - today()).days) for offset in info['EffExp']],
    tenor_days=[round(((today() + parse_offset(offset)) - today()).days) for offset in info['MtyTnr']]
)

In [ ]:
rates = irbt.hist.loc[:, (slice(None), 'rate')].droplevel(1, axis=1).set_axis(info['EffExp'].astype('str') + 'x' + info['MtyTnr'].astype('str'), axis=1) * 100

In [ ]:
dv01 = irbt.hist.loc[:, (slice(None), 'ann01')].droplevel(1, axis=1).set_axis(info['EffExp'].astype('str') + 'x' + info['MtyTnr'].astype('str'), axis=1) * 100

In [ ]:
X = rates.ffill().diff().dropna()
# X = rates.ffill().dropna()

In [ ]:
X = rates.ffill().rolling(10).mean().dropna()
# X = X.diff().dropna()

In [ ]:
res = []
step = 90
for j, i in enumerate(range(0, len(X), step)):
    

    
    X_ = X.iloc[i: i + step]
    if X_.empty:
        continue
    pca_ = PCA(n_components=3)
    pca_.fit(X_)
    
    if j == 0:
        sign = np.sign(pca_.components_[:, -1])
    sign_ = sign * np.sign(pca_.components_[:, -1])    
    # sign_ = sign_ * 0 + 1
        
    F_ = pca_.transform(X_) * sign_
    eig_ = pca_.components_ * sign_[:, None]
    
    res.append(
        {
            'pca': pca_, 
            'X': X_,
            'F': F_,
            'eig': eig_,
            'ev': pca_.explained_variance_ratio_,
            'n': len(X_)
        }
    )

In [ ]:
for r in res:
    print(f"{r['X'].index[0]:%b%Y} - {r['X'].index[-1]:%b%Y}")
    display(pd.DataFrame(r['eig'], columns=[f"{a}x{b}" for a, b in default_expiries_tenors]).T.plot())

In [ ]:
pd.concat([r['F'].pct_change() for r in res])

In [ ]:
pd.concat([r['F'] for r in res]).cumsum().plot()

In [ ]:
pca = PCA(n_components=3)

In [ ]:
pca.fit(X[-250:])

In [ ]:
F = pca.transform(X)

In [ ]:
F.plot()

In [ ]:
F.plot()

In [ ]:
pd.concat([F['pca0'].diff(), X.diff()], axis=1).corr()

In [ ]:
X - X.T

In [ ]:
def get_spreads_from_rates(X):
    arr = X.to_numpy()
    diff3 = (arr.T[:, None] - arr.T).T
    spreads = pd.DataFrame(diff3.reshape(-1, arr.shape[1]**2), columns=pd.MultiIndex.from_product([X.columns, X.columns]).reorder_levels([1, 0]), index=X.index)\
        .iloc[:, [*chain.from_iterable([range(i + X.shape[1]*(i - 1), X.shape[1] * i) for i in range(1, X.shape[1])])]]
    return spreads

In [ ]:
get_spreads_from_rates(rates).iloc[-1].unstack()

In [ ]:
arr = X.to_numpy()

In [ ]:
diff3 = (arr.T[:, None] - arr.T).T

In [ ]:
pd.DataFrame(diff3.reshape(-1, arr.shape[1]**2), columns=pd.MultiIndex.from_product([X.columns, X.columns]).reorder_levels([1, 0]))

In [ ]:
spreads = pd.DataFrame(diff3.reshape(-1, arr.shape[1]**2), columns=pd.MultiIndex.from_product([X.columns, X.columns]).reorder_levels([1, 0]), index=X.index)\
    .iloc[:, [*chain.from_iterable([range(i + X.shape[1]*(i - 1), X.shape[1] * i) for i in range(1, X.shape[1])])]]

In [ ]:
spreads.iloc[-1].unstack()

In [ ]:
(arr.shape[1] ** 2 - arr.shape[1]) / 2

In [ ]:
[*chain.from_iterable([range(i + X.shape[1]*i - 1, X.shape[1] * i) for i in range(1, X.shape[1])])]

In [ ]:
pd.Series(diff3[:, 0, 3])

In [ ]:
X.iloc[:, 3] - X.iloc[:, 0]

In [ ]:
pd.concat([F['pca1'].diff(), X.diff()], axis=1).corr()

In [ ]:
pd.DataFrame(pca.components_, columns=[f"{a}x{b}" for a, b in default_expiries_tenors]).T.plot()

In [ ]:
pca.components_

In [ ]:
mean = X.mean()

In [ ]:
X0 = X - mean

In [ ]:
(X0 @ pca.components_.T).plot()

In [ ]:
pd.DataFrame(pca.components_).T.plot()

In [ ]:
pd.DataFrame(pca.components_, columns=X.columns)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
X = rates.ffill().dropna()

In [ ]:
pca = PCA(n_components=None)

In [ ]:
pca.fit(X)

In [ ]:
F = pca.transform(X)

In [ ]:
F.plot()

In [ ]:
pd.DataFrame(pca.components_).T.plot()

In [ ]:
F.plot()

In [ ]:
pca.explained_variance_ratio_